In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=1)  # Reduced filters
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 10, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(10)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(10, 20, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(20)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout(0.25)
        self.fc = nn.Linear(20*7*7, 10)  # Fully connected layer

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(F.relu(self.bn2(self.conv2(x))))
        x = self.pool2(F.relu(self.bn3(self.conv3(x))))
        x = self.drop(x)
        x = x.view(-1, 20*7*7)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
       BatchNorm2d-2           [-1, 10, 28, 28]              20
            Conv2d-3           [-1, 10, 28, 28]             910
       BatchNorm2d-4           [-1, 10, 28, 28]              20
         MaxPool2d-5           [-1, 10, 14, 14]               0
            Conv2d-6           [-1, 20, 14, 14]           1,820
       BatchNorm2d-7           [-1, 20, 14, 14]              40
         MaxPool2d-8             [-1, 20, 7, 7]               0
           Dropout-9             [-1, 20, 7, 7]               0
           Linear-10                   [-1, 10]           9,810
Total params: 12,720
Trainable params: 12,720
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.33
Params size (MB): 0.05
Estimated Tot

In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

100%|██████████| 9912422/9912422 [00:00<00:00, 109793755.65it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 76960415.39it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31257983.64it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 13129241.05it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 19):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.028060143813490868 batch_id=468: 100%|██████████| 469/469 [01:31<00:00,  5.10it/s]



Test set: Average loss: 0.0818, Accuracy: 9747/10000 (97%)



loss=0.022594952955842018 batch_id=468: 100%|██████████| 469/469 [01:19<00:00,  5.90it/s]



Test set: Average loss: 0.0437, Accuracy: 9855/10000 (99%)



loss=0.09878575056791306 batch_id=468: 100%|██████████| 469/469 [01:21<00:00,  5.74it/s]


...

loss=0.009853041730821133 batch_id=468: 100%|██████████| 469/469 [01:19<00:00,  5.89it/s]



Test set: Average loss: 0.0308, Accuracy: 9887/10000 (99%)



loss=0.07644367963075638 batch_id=468: 100%|██████████| 469/469 [01:19<00:00,  5.87it/s]



Test set: Average loss: 0.0348, Accuracy: 9884/10000 (99%)



loss=0.010795511305332184 batch_id=419:  90%|████████▉ | 420/469 [01:10<00:07,  6.68it/s]